In [32]:
import time
import numpy as np
import pandas as pd
from os.path import join
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

folder = 'data'
df = pd.read_excel(join(folder, '10.25.14 us and uk 900.xlsx'))
df = df[['[Dx] Diagnosis Chosen'] + [c for c in df if c not in ['[Dx] Diagnosis Chosen']]]  # move class to first column
df['[Dx] Diagnosis Chosen'].fillna('None', inplace=True)  # fill class column Nan with 'None'
df = df.drop('[Initial_Dx] ', 1)  # drop Initial_Dx column
x = df.iloc[:, 1:]
y = df.iloc[:, 0]
#df.head()
#df.columns

In [33]:
def preprocess(x, y):
    features = list(x.columns.values)
    discrete_features = []
    for f in features:
        if x[f].dtype == np.object:
            discrete_features.append(f)
    x[discrete_features] = x[discrete_features].astype(str)  # convert dtype 'object' to 'str'
    # one hot encoding for categorical feature values
    x_dummies = pd.get_dummies(x, columns=discrete_features, dummy_na=True)  
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')  # convert NaN to column mean value
    x_new = imp_mean.fit_transform(x_dummies)  
    return x_new, y
start = time.time()
x_new, y_new = preprocess(x, y)
rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)  # n_estimators: # trees, n_jobs -1 all cpu cores
accuracy = cross_val_score(rf, x_new, y_new, cv=5, scoring='accuracy')  # k-fold cross validation, k=cv
acc = accuracy.mean()
print('accuracy: ', acc)
print('--- running time: %.4f seconds ---' % (time.time() - start))

accuracy:  0.8250532452427025
--- running time: 1.1451 seconds ---


In [23]:
classes = np.unique(y)
classes

array(['None', 'ankylosing', 'psoriatic', 'rheumatoid', 'sjogren',
       'still', 'systemic'], dtype=object)